## Gibbs sampling on distributions with quasi deterministic random variables

Assume a binary logical operation represented by a Bayesian Network $P(A,B,O)$ that factores into the distributions $P(A)$, $P(B)$, $P(O|A,B)$. Using Gibbs sampling to approxmimate joint distribution $P(A,B)$, we need to define the resampling distributions 

\begin{align}
P(A|mb(A)) &= 1/Z P(A)P(O|A,B) \\
P(B|mb(B)) &= 1/Z P(B)P(O|A,B) \\
P(O|mb(O)) &= 1/Z P(O|A,B)
\end{align}


If the operation $O$ is deterministic on the inputs $A$ and $B$ then Gibbs sampling might get stuck in the initial sample configuration as can be seen in the following. Assume the distributions to be given by

| A |P(A)|
| --|:---:|
| 0 | 0.5 |
| 1 | 0.5 |

| B | P(B)|
| --|:-----:|
| 0 | 0.5 |
| 1 | 0.5 |

| AB| P(O-/A,B)|P(O+/A,B)|
| --|:--------:|:-----:|
| 00 | 1.0 | 0.0 |
| 01 | 0.0 | 1.0 |
| 10 | 0.0 | 1.0 |
| 11 | 1.0 | 0.0 |

which is the classical XOR gate with uniform random inputs.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
A = np.array([0.5, 0.5])
B = np.array([0.5, 0.5])
O_AB = np.array([[[1.0, 0.0], [0.0, 1.0]], [[0.0, 1.0], [1.0, 0.0]]])

Next, define the resample distributions

In [4]:
def redist_A():
    A_mb_ABO = A[:, None, None] * O_AB.transpose((2,0,1)) # Read: resample dist of A using variables A,B,O in that order
    A_mb_ABO /= A_mb_ABO.sum(axis=0)[None,...]
    return A_mb_ABO

redist_A()

array([[[ 1.,  0.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

In [5]:
def redist_B():
    B_mb_BAO = B[:, None, None] * O_AB.transpose((2,1,0))
    B_mb_BAO /= B_mb_BAO.sum(axis=0)[None,...]
    return B_mb_BAO

redist_B()

array([[[ 1.,  0.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

In [6]:
def redist_O():
    O_mb_OAB = O_AB
    return O_mb_OAB

redist_O()

array([[[ 1.,  0.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 1.,  0.]]])

What these resampling distributions indicate is that they are all fully deterministic, despite the marginal distributions of A, B being probabilistic. Hence, the value of any of these variables is fully determined by the known value of the operation and the other second variable. While forward sampling does the right job in this case, Gibbs fails as shown below

In [7]:
def draw(choices, probs, n=None):
    '''Draw samples from probability distributions.
    
    If n is given, n samples from the same distribution will be drawn.
    If n is not given, the number of samples is determined by the number
    of columns in probs. Each column then represents a unique probability
    distribution to draw a single sample from.
    '''
    
    if n is not None:
        u = np.random.uniform(0., 1., size=n)
        c = np.cumsum(probs)
        return np.asarray(choices)[c.searchsorted(u)]
    else:
        u = np.random.uniform(0., 1., size=probs.shape[1])
        c = np.cumsum(probs, axis=0)
        return np.asarray([c[:,i].searchsorted(u[i]) for i in range(probs.shape[1])])

# self test for draw
assert np.isclose(draw([0, 1], [0.3, 0.7], 10000).sum()/10000, 0.7, atol=1e-2)
assert np.isclose(draw([0, 1], np.tile([[0.3], [0.7]], 10000)).sum()/10000, 0.7, atol=1e-2)

In [8]:
def forward_sample(n=1000):
    '''Approximates P(A,B) using forward sampling.'''
    
    # Draw samples through forward sampling
    samples = np.zeros((n, 3), dtype=np.int32)
    samples[:, 0] = draw([0, 1], A, n=n)
    samples[:, 1] = draw([0, 1], B, n=n)
    samples[:, 2] = draw([0, 1], O_AB[:, samples[:, 0], samples[:, 1]])
    
    # Approximate P(A,B) by counting
    return np.array([
        [((samples[:, 0] == 0) & (samples[:, 1] == 0)).sum(), ((samples[:, 0] == 0) & (samples[:, 1] == 1)).sum()],
        [((samples[:, 0] == 1) & (samples[:, 1] == 0)).sum(), ((samples[:, 0] == 1) & (samples[:, 1] == 1)).sum()]]) / n

np.array([forward_sample(n=1000) for _ in range(100)]).mean(axis=0) # 100 experiments, each 1000 samples, mean plotted

array([[ 0.24762,  0.25254],
       [ 0.25157,  0.24827]])

Forward sampling produces expected results, each combination of A,B occurs equally likely. Exceptional: not looking at children is a benefit. Whereas Gibbs sampling is stuck

In [9]:
def gibbs_sample(n=1000, burn_in=100, dist=1, start=None):
    choices = [0, 1]
    
    A_mb_ABO = redist_A()
    B_mb_BAO = redist_B()
    O_mb_OAB = redist_O()
    
    resample = [
        lambda x: draw(choices, A_mb_ABO[:, x[1], x[2]], n=1), # A
        lambda x: draw(choices, B_mb_BAO[:, x[0], x[2]], n=1), # B
        lambda x: draw(choices, O_mb_OAB[:, x[0], x[1]], n=1), # O
    ]
    
    def gibbs_update(x, c):
        x[c] = resample[c](x)
        return x
    
    # Which vars: A, B, C, A, ...
    seq_ids = np.zeros(n*dist+burn_in, dtype=np.int32)
    seq_ids[1::3] = 1
    seq_ids[2::3] = 2
    
    if start is None:
        x = np.array([0, 1, 1], dtype=np.int32) # Start vec(A,B,C)
    else:
        x = np.asarray(start)
    
    idx = 0
    def step():
        nonlocal idx
        v = seq_ids[idx]
        x[v] = resample[v](x)
        idx += 1
        return x

    def advance(n):
        for _ in range(n):
            step()

    # burn-in
    advance(burn_in)
    
    # sample
    samples = np.empty((n, 3))
    for i in range(n):
        samples[i] = step()
        advance(dist-1)
                
    # Approximate P(A,B) by counting
    return np.array([
        [((samples[:, 0] == 0) & (samples[:, 1] == 0)).sum(), ((samples[:, 0] == 0) & (samples[:, 1] == 1)).sum()],
        [((samples[:, 0] == 1) & (samples[:, 1] == 0)).sum(), ((samples[:, 0] == 1) & (samples[:, 1] == 1)).sum()]]) / n
        
# No matter which start point we always produce the same samples, so P(A,B) for all particular start possibilities
from IPython.display import display
display(gibbs_sample(burn_in=10, dist=10, n=1000, start=[0,0,0]))
display(gibbs_sample(burn_in=10, dist=10, n=1000, start=[0,1,1]))
display(gibbs_sample(burn_in=10, dist=10, n=1000, start=[1,0,1]))
display(gibbs_sample(burn_in=10, dist=10, n=1000, start=[1,1,0]))

array([[ 1.,  0.],
       [ 0.,  0.]])

array([[ 0.,  1.],
       [ 0.,  0.]])

array([[ 0.,  0.],
       [ 1.,  0.]])

array([[ 0.,  0.],
       [ 0.,  1.]])

No matter what's the starting condition of the Markov chain, we remain in that state forever.

By making $P(O|A,B)$ quasi-determinstic we can slowly see Gibbs sampling doing the right thing as far as $P(A,B)$ is concerned. 

In [11]:
O_AB[:] = np.array([[[0.99, 0.01], [0.01, 0.99]], [[0.01, 0.99], [0.99, 0.01]]])
display(gibbs_sample(burn_in=100, dist=10, n=1000, start=[0,0,0]))

array([[ 0.215,  0.234],
       [ 0.263,  0.288]])

A bit more..

In [12]:
O_AB[:] = np.array([[[0.9, 0.1], [0.1, 0.9]], [[0.1, 0.9], [0.9, 0.1]]])
display(gibbs_sample(burn_in=100, dist=10, n=1000, start=[0,0,0]))

array([[ 0.236,  0.258],
       [ 0.242,  0.264]])